In [0]:
# ==========================================
#  API Orders Analytics - Script Completo
# ==========================================

import requests
import urllib3
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# ------------------------------------------
# 1. Consumir API (deshabilitar SSL)
# ------------------------------------------

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://ca0589a4-4f03-493a-bd52-56f4afeb7625.mock.pstmn.io/orders"

response = requests.get(url, verify=False)
orders = response.json()

# ------------------------------------------
# 2. Crear DataFrame Spark
# ------------------------------------------

df = spark.createDataFrame(orders)

print("Total órdenes:", df.count())
df.printSchema()
display(df)



Total órdenes: 1000
root
 |-- cantidad: long (nullable = true)
 |-- category: string (nullable = true)
 |-- cliente_id: long (nullable = true)
 |-- direccion_envio: string (nullable = true)
 |-- fecha_compra: string (nullable = true)
 |-- metodo_pago: string (nullable = true)
 |-- moneda: string (nullable = true)
 |-- nombre_producto: string (nullable = true)
 |-- orden_id: long (nullable = true)
 |-- pais_envio: string (nullable = true)
 |-- precio_total: double (nullable = true)
 |-- precio_unitario: double (nullable = true)



cantidad,category,cliente_id,direccion_envio,fecha_compra,metodo_pago,moneda,nombre_producto,orden_id,pais_envio,precio_total,precio_unitario
10,Food - Bakery,705239,670 Sherman Court,11/17/2023,PayPal,NIO,Maple Pecan Oatmeal Cookies,1,Nicaragua,117403.9,11740.39
3,Outdoor,425030,4793 Homewood Alley,3/17/2020,Debito,CRC,Portable Folding Picnic Table,2,Costa Rica,1511911.17,503970.39
17,Toys,348063,2979 Hauk Alley,10/23/2022,Debito,NIO,Kids' Science Experiment Kit,3,Nicaragua,846227.53,49778.09
8,Electronics,379212,6 Fallview Circle,4/19/2020,Apple Pay,HNL,High-Speed HDMI Cable,4,Honduras,117253.6,14656.7
4,Travel,725283,74692 Crownhardt Terrace,5/13/2021,Apple Pay,PAB,Overnight Duffle Bag,5,Panama,3386.96,846.74
8,Travel,472011,9 Clyde Gallagher Drive,2/4/2024,PayPal,HNL,Travel Sewing Kit,6,Honduras,243289.84,30411.23
3,Food - Snacks,936072,79 Waubesa Hill,6/20/2023,Credito,USD,Chocolate Coconut Protein Balls,7,El Salvador,2046.21,682.07
2,Food - Dairy Alternatives,413470,3 Tennessee Park,3/22/2020,PayPal,CRC,Coconut Milk Yogurt,8,Costa Rica,463144.98,231572.49
5,Food - Breakfast,377846,281 Arizona Road,12/7/2021,PayPal,CRC,Cranberry Orange Oatmeal,9,Costa Rica,410373.4,82074.68
8,Crafts,580829,9 Beilfuss Circle,4/1/2021,Apple Pay,NIO,Electric Heat Press Machine,10,Nicaragua,380008.8,47501.1


In [0]:

# ------------------------------------------
# 3. Tabla de Tipo de Cambio
# ------------------------------------------

fx_rates = [
    ("El Salvador", "USD", 1.0),
    ("Panama",      "PAB", 1.0),
    ("Costa Rica",  "CRC", 0.0019),
    ("Nicaragua",   "NIO", 0.027),
    ("Honduras",    "HNL", 0.041),
    ("Guatemala",   "GTQ", 0.13)
]

df_fx = spark.createDataFrame(fx_rates, ["pais_envio", "moneda", "rate_to_usd"])


# ------------------------------------------
# 4. Join + convertir precio_total a USD
# ------------------------------------------

df_final = (
    df.join(df_fx, ["pais_envio", "moneda"], "left")
      .withColumn("precio_total_usd", F.col("precio_total") * F.col("rate_to_usd"))
)

print("=== Data Normalizada con USD ===")
display(df_final)



=== Data Normalizada con USD ===


pais_envio,moneda,cantidad,category,cliente_id,direccion_envio,fecha_compra,metodo_pago,nombre_producto,orden_id,precio_total,precio_unitario,rate_to_usd,precio_total_usd
Nicaragua,NIO,10,Food - Bakery,705239,670 Sherman Court,11/17/2023,PayPal,Maple Pecan Oatmeal Cookies,1,117403.9,11740.39,0.027,3169.9053
Costa Rica,CRC,3,Outdoor,425030,4793 Homewood Alley,3/17/2020,Debito,Portable Folding Picnic Table,2,1511911.17,503970.39,0.0019,2872.631223
Nicaragua,NIO,17,Toys,348063,2979 Hauk Alley,10/23/2022,Debito,Kids' Science Experiment Kit,3,846227.53,49778.09,0.027,22848.14331
Honduras,HNL,8,Electronics,379212,6 Fallview Circle,4/19/2020,Apple Pay,High-Speed HDMI Cable,4,117253.6,14656.7,0.041,4807.3976
Panama,PAB,4,Travel,725283,74692 Crownhardt Terrace,5/13/2021,Apple Pay,Overnight Duffle Bag,5,3386.96,846.74,1.0,3386.96
Honduras,HNL,8,Travel,472011,9 Clyde Gallagher Drive,2/4/2024,PayPal,Travel Sewing Kit,6,243289.84,30411.23,0.041,9974.88344
El Salvador,USD,3,Food - Snacks,936072,79 Waubesa Hill,6/20/2023,Credito,Chocolate Coconut Protein Balls,7,2046.21,682.07,1.0,2046.21
Costa Rica,CRC,2,Food - Dairy Alternatives,413470,3 Tennessee Park,3/22/2020,PayPal,Coconut Milk Yogurt,8,463144.98,231572.49,0.0019,879.975462
Costa Rica,CRC,5,Food - Breakfast,377846,281 Arizona Road,12/7/2021,PayPal,Cranberry Orange Oatmeal,9,410373.4,82074.68,0.0019,779.70946
Nicaragua,NIO,8,Crafts,580829,9 Beilfuss Circle,4/1/2021,Apple Pay,Electric Heat Press Machine,10,380008.8,47501.1,0.027,10260.2376


In [0]:
# ------------------------------------------
# 5. RESUMEN POR PAÍS
# ------------------------------------------

df_country = (
    df_final.groupBy("pais_envio")
            .agg(
                F.count("orden_id").alias("total_ordenes"),
                F.sum("precio_total_usd").alias("monto_total_usd")
            )
            .orderBy("pais_envio")
)

print("=== RESUMEN POR PAÍS ===")
display(df_country)

=== RESUMEN POR PAÍS ===


pais_envio,total_ordenes,monto_total_usd
Costa Rica,126,823804.7781419997
El Salvador,77,643513.93
Guatemala,235,1863072.6634
Honduras,297,2558834.81402
Nicaragua,112,908675.6995799998
Panama,153,1267353.7900000005


In [0]:
# ------------------------------------------
# 6. TOP 3 CATEGORÍAS POR PAÍS
# ------------------------------------------

df_cat = (
    df_final.groupBy("pais_envio", "category")
            .agg(
                F.sum("cantidad").alias("cantidad_total"),
                F.sum("precio_total_usd").alias("monto_total_usd")
            )
)

window_rank = Window.partitionBy("pais_envio").orderBy(F.desc("cantidad_total"))

df_top3 = (
    df_cat.withColumn("rank", F.row_number().over(window_rank))
          .filter(F.col("rank") <= 3)
          .orderBy("pais_envio", F.desc("cantidad_total"))
)

print("=== TOP 3 CATEGORÍAS POR PAÍS ===")
display(df_top3)


# ------------------------------------------
# Fin del script
# ------------------------------------------



=== TOP 3 CATEGORÍAS POR PAÍS ===


pais_envio,category,cantidad_total,monto_total_usd,rank
Costa Rica,Food - Snacks,124,75563.361437,1
Costa Rica,Pets,75,59459.103233999995,2
Costa Rica,Food - Condiments,73,47270.99528,3
El Salvador,Kitchen,79,57176.41,1
El Salvador,Food - Condiments,58,37612.240000000005,2
El Salvador,Home,47,48722.1,3
Guatemala,Food - Snacks,203,157906.0626,1
Guatemala,Food - Condiments,197,158052.39969999998,2
Guatemala,Kitchen,180,101627.11,3
Honduras,Food - Snacks,334,238350.93135,1
